<h1 align="center">Advanced Registration</h1>


**Summary:**
1. SimpleITK provides two flavors of non-rigid registration:
   * Free Form Deformation, BSpline based, and Demons using the ITKv4 registration framework.
   * A set of Demons filters that are independent of the registration framework (`DemonsRegistrationFilter, DiffeomorphicDemonsRegistrationFilter, FastSymmetricForcesDemonsRegistrationFilter, SymmetricForcesDemonsRegistrationFilter`).
2. Registration evaluation:
   * Registration accuracy, the quantity of interest is the Target Registration Error (TRE).
   * TRE is spatially variant.
   * Surrogate metrics for evaluating registration accuracy such as segmentation overlaps are relevant, but are potentially deficient.
   * Registration time.
   * Acceptable values for TRE and runtime are context dependent.

In [ ]:
library(SimpleITK)
library(ggplot2)
library(tidyr)
library(plot3D)
library(purrr)

source("downloaddata.R")
source("registration_gui.R")
source("utilities.R")

## Data and Registration Task

In this notebook we will use the Point-validated Pixel-based Breathing Thorax Model (POPI). This is a 4D (3D+time) thoracic-abdominal CT (10 CTs representing the respiratory cycle) with masks segmenting each of the CTs to air/body/lung, and a set of corresponding landmarks localized in each of the CT volumes.

The registration problem we deal with is non-rigid alignment of the lungs throughout the respiratory cycle. This information is relevant for radiation therapy planning and execution.


The POPI model is provided by the Léon Bérard Cancer Center & CREATIS Laboratory, Lyon, France. The relevant publication is:

J. Vandemeulebroucke, D. Sarrut, P. Clarysse, "The POPI-model, a point-validated pixel-based breathing thorax model",
Proc. XVth International Conference on the Use of Computers in Radiation Therapy (ICCR), Toronto, Canada, 2007.

Additional 4D CT data sets with reference points are available from the CREATIS Laboratory <a href="http://www.creatis.insa-lyon.fr/rio/popi-model?action=show&redirect=popi">here</a>. 

In [ ]:
image_file_names <- file.path("POPI", "meta", paste0(c(0,7), "0-P.mhd"))
# Read the CT images as 32bit float, the pixel type required for registration.
images <- lapply(image_file_names, function(image_file_name) ReadImage(fetch_data(image_file_name), "sitkFloat32"))    

mask_file_names <- file.path("POPI", "masks", paste0(c(0,7), "0-air-body-lungs.mhd"))
masks <- lapply(mask_file_names, function(mask_file_name) ReadImage(fetch_data(mask_file_name)))    


points_file_names <- file.path("POPI", "landmarks", paste0(c(0,7), "0-Landmarks.pts"))
points <- lapply(points_file_names, function(points_file_name) read.table(fetch_data(points_file_name)))

# Look at a temporal slice for the mid coronal index     
coronal_index <- as.integer(round(images[[1]]$GetHeight()/2.0))
temporal_slice <- temporal_coronal_with_overlay(coronal_index, images, masks, popi_lung_label, -1024, 976)
    # Flip the image so that it corresponds to the standard radiological display.
Show(temporal_slice[,seq(temporal_slice$GetHeight(),0,-1),]) 

## Free Form Deformation

Define a BSplineTransform using a sparse set of grid points overlaid onto the fixed image's domain to deform it.

For the current registration task we are fortunate in that we have a unique setting. The images are of the same patient during respiration so we can initialize the registration using the identity transform. Additionally, we have masks demarcating the lungs.

We use the registration framework taking advantage of its ability to use masks that limit the similarity metric estimation to points lying inside our region of interest, the lungs.

In [ ]:
fixed_index = 1
moving_index = 2

fixed_image <- images[[fixed_index]]
fixed_image_mask <- masks[[fixed_index]] == popi_lung_label
fixed_points <- t(as.matrix(points[[fixed_index]]))

moving_image <- images[[moving_index]]
moving_image_mask <- masks[[moving_index]] == popi_lung_label
moving_points <- t(as.matrix(points[[moving_index]]))

In [ ]:
registration_method <- ImageRegistrationMethod()
    
# Determine the number of Bspline control points using the physical spacing we want for the control grid. 
grid_physical_spacing <- c(50.0, 50.0, 50.0) # A control point every 50mm
image_physical_size <- fixed_image$GetSize() * fixed_image$GetSpacing()
mesh_size <- as.integer(round(image_physical_size/grid_physical_spacing))

initial_transform <- BSplineTransformInitializer(image1 = fixed_image, 
                                                 transformDomainMeshSize = mesh_size, order=3)    
registration_method$SetInitialTransform(initial_transform)
        
registration_method$SetMetricAsMeanSquares()
registration_method$SetMetricSamplingStrategy("RANDOM")
registration_method$SetMetricSamplingPercentage(0.01)
registration_method$SetMetricFixedMask(fixed_image_mask)

registration_method$SetShrinkFactorsPerLevel(shrinkFactors = c(4,2,1))
registration_method$SetSmoothingSigmasPerLevel(smoothingSigmas=c(2,1,0))
registration_method$SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

registration_method$SetInterpolator("sitkLinear")
registration_method$SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5, numberOfIterations=100)
        
final_transformation <- registration_method$Execute(fixed_image, moving_image)
cat(paste0("Optimizer\'s stopping condition, ",registration_method$GetOptimizerStopConditionDescription()))
cat("\n","With metric value: ", registration_method$GetMetricValue())

## Qualitative evaluation via segmentation transfer

Transfer the segmentation from the moving image to the fixed image before and after registration and visually evaluate overlap.

In [ ]:
transformed_segmentation <- Resample(masks[[moving_index]],
                                     fixed_image,
                                     final_transformation, 
                                     "sitkNearestNeighbor",
                                      0.0, 
                                      moving_image_mask$GetPixelID())

temporal_slice <- temporal_coronal_with_overlay(coronal_index, c(fixed_image,fixed_image), 
                                                c(masks[[moving_index]], transformed_segmentation), 
                                                popi_lung_label, -1024, 976)
    # Flip the image so that it corresponds to the standard radiological display.
Show(temporal_slice[,seq(temporal_slice$GetHeight(),0,-1),]) 

### Quantitative evaluation 

The most appropriate evaluation is based on analysis of Target Registration Errors(TRE), which is defined as follows:

Given the transformation $T_f^m$ and corresponding points in the two coordinate systems, $^fp,^mp$, points which were not used in the registration process, TRE is defined as $\|T_f^m(^fp) - ^mp\|$. 

We start by looking at some descriptive statistics of TRE:

In [ ]:
initial_TRE <- target_registration_errors(Transform(), fixed_points, moving_points)
final_TRE <- target_registration_errors(final_transformation, fixed_points, moving_points)

cat(paste0('Initial alignment errors in millimeters, mean(std): ',
           sprintf('%.2f',mean(initial_TRE)),'(',sprintf('%.2f',sd(initial_TRE)),') max:', sprintf('%.2f\n',max(initial_TRE))))
cat(paste0('Final alignment errors in millimeters, mean(std): ',
           sprintf('%.2f',mean(final_TRE)),'(',sprintf('%.2f',sd(final_TRE)),') max:', sprintf('%.2f\n',max(final_TRE))))

The above descriptive statistics do not convey the whole picture, we should also look at the TRE distributions before and after registration.

In [ ]:
df <- data.frame(AfterRegistration=final_TRE, BeforeRegistration=initial_TRE)
df.long <- gather(df, key=ErrorType, value=ErrorMagnitude)

ggplot(df.long, aes(x=ErrorMagnitude, group=ErrorType, colour=ErrorType, fill=ErrorType)) + 
geom_histogram(bins=20,position='identity', alpha=0.3) + 
theme(legend.title=element_blank(), legend.position=c(.85, .85))
## Or, if you prefer density plots
ggplot(df.long, aes(x=ErrorMagnitude, group=ErrorType, colour=ErrorType, fill=ErrorType)) + 
geom_density(position='identity', alpha=0.3) + 
theme(legend.title=element_blank(), legend.position=c(.85, .85))

Finally, we should also take into account the fact that TRE is spatially variant (think center of rotation). We therefore should also explore the distribution of errors as a function of the point location.

In [ ]:
max_TRE = max(initial_TRE, final_TRE)

colors <- colorRampPalette(c("black", "red", "yellow", "white"))(length(fixed_points))

scatter3D(fixed_points[1,], fixed_points[2,], fixed_points[3,],
          main="TRE (mm) before registration",
          pch = 19, colvar=initial_TRE, col = colors, clim=c(0,max_TRE))

scatter3D(fixed_points[1,], fixed_points[2,], fixed_points[3,],
          main="TRE (mm) after registration",
          pch = 19, colvar=final_TRE, col = colors, clim=c(0,max_TRE))

Deciding whether a registration algorithm is appropriate for a specific problem is context dependent and is defined by the clinical/research needs both in terms of accuracy and computational complexity.

## Demons Based Registration

SimpleITK includes a number of filters from the Demons registration family (originally introduced by J. P. Thirion):
1. DemonsRegistrationFilter.
2. DiffeomorphicDemonsRegistrationFilter.
3. FastSymmetricForcesDemonsRegistrationFilter.
4. SymmetricForcesDemonsRegistrationFilter.

These are appropriate for mono-modal registration. As these filters are independent of the ImageRegistrationMethod we do not have access to the multiscale framework. Luckily it is easy to implement our own multiscale framework in SimpleITK, which is what we do in the next cell.

In [ ]:
#    
# Args:
#        image: The image we want to resample.
#        shrink_factor: A number greater than one, such that the new image's size is original_size/shrink_factor.
#        smoothing_sigma: Sigma for Gaussian smoothing, this is in physical (image spacing) units, not pixels.
#    Return:
#        Image which is a result of smoothing the input and then resampling it using the given sigma and shrink factor.
#
smooth_and_resample <- function(image, shrink_factor, smoothing_sigma)
{
    smoothed_image <- SmoothingRecursiveGaussian(image, smoothing_sigma)
    
    original_spacing <- image$GetSpacing()
    original_size <- image$GetSize()
    new_size <-  as.integer(round(original_size/shrink_factor))
    new_spacing <- (original_size-1)*original_spacing/(new_size-1)

    return(Resample(smoothed_image, new_size, Transform(), 
                    "sitkLinear", image$GetOrigin(),
                    new_spacing, image$GetDirection(), 0.0, 
                    image$GetPixelID()))
}

#    
# Run the given registration algorithm in a multiscale fashion. The original scale should not be given as input as the
# original images are implicitly incorporated as the base of the pyramid.
# Args:
#   registration_algorithm: Any registration algorithm that has an Execute(fixed_image, moving_image, displacement_field_image)
#                           method.
#   fixed_image: Resulting transformation maps points from this image's spatial domain to the moving image spatial domain.
#   moving_image: Resulting transformation maps points from the fixed_image's spatial domain to this image's spatial domain.
#   initial_transform: Any SimpleITK transform, used to initialize the displacement field.
#   shrink_factors: Shrink factors relative to the original image's size.
#   smoothing_sigmas: Amount of smoothing which is done prior to resampling the image using the given shrink factor. These
#                     are in physical (image spacing) units.
# Returns: 
#    DisplacementFieldTransform
#
multiscale_demons <- function(registration_algorithm, fixed_image, moving_image, initial_transform = NULL, 
                              shrink_factors=NULL, smoothing_sigmas=NULL)
{    
    # Create image pyramids. 
    fixed_images <- c(fixed_image, 
                      if(!is.null(shrink_factors))
                          map2(rev(shrink_factors), rev(smoothing_sigmas), 
                               ~smooth_and_resample(fixed_image, .x, .y))
                      )
    moving_images <- c(moving_image, 
                       if(!is.null(shrink_factors))
                           map2(rev(shrink_factors), rev(smoothing_sigmas), 
                               ~smooth_and_resample(moving_image, .x, .y))
                       )

    # Uncomment the following two lines if you want to see your image pyramids.
    #lapply(fixed_images, Show)
    #lapply(moving_images, Show)
    
                              
    # Create initial displacement field at lowest resolution. 
    # Currently, the pixel type is required to be sitkVectorFloat64 because of a constraint imposed by the Demons filters.
    lastImage <- fixed_images[[length(fixed_images)]]
    if(!is.null(initial_transform))
    {
        initial_displacement_field = TransformToDisplacementField(initial_transform, 
                                                                  "sitkVectorFloat64",
                                                                  lastImage$GetSize(),
                                                                  lastImage$GetOrigin(),
                                                                  lastImage$GetSpacing(),
                                                                  lastImage$GetDirection())
    }
    else
    {
        initial_displacement_field <- Image(lastImage$GetWidth(), 
                                            lastImage$GetHeight(),
                                            lastImage$GetDepth(),
                                            "sitkVectorFloat64")
        initial_displacement_field$CopyInformation(lastImage)
    }
    # Run the registration pyramid, run a registration at the top of the pyramid and then iterate: 
    # a. resampling previous deformation field onto higher resolution grid.
    # b. register.
    initial_displacement_field <- registration_algorithm$Execute(fixed_images[[length(fixed_images)]], 
                                                                moving_images[[length(moving_images)]], 
                                                                initial_displacement_field)
    # This is a use case for a loop, because the operations depend on the previous step. Otherwise
    # we need to mess around with tricky assignments to variables in different scopes
    for (idx in seq(length(fixed_images)-1,1)) {
        f_image <- fixed_images[[idx]]
        m_image <- moving_images[[idx]]
        initial_displacement_field <- Resample(initial_displacement_field, f_image)
        initial_displacement_field <- registration_algorithm$Execute(f_image, m_image, initial_displacement_field)
    }

    return(DisplacementFieldTransform(initial_displacement_field))
}

Now we will use our newly minted multiscale framework to perform registration with the Demons filters. Some things you can easily try out by editing the code below:
1. Is there really a need for multiscale - just call the multiscale_demons method without the shrink_factors and smoothing_sigmas parameters.
2. Which Demons filter should you use - configure the other filters and see if our selection is the best choice (accuracy/time).

In [ ]:
# Select a Demons filter and configure it.
demons_filter <-  FastSymmetricForcesDemonsRegistrationFilter()
demons_filter$SetNumberOfIterations(20)
# Regularization (update field - viscous, total field - elastic).
demons_filter$SetSmoothDisplacementField(TRUE)
demons_filter$SetStandardDeviations(2.0)

# Run the registration.
tx <- multiscale_demons(registration_algorithm=demons_filter, 
                       fixed_image, 
                       moving_image,
                       shrink_factors = c(4,2),
                       smoothing_sigmas = c(8,4))

# Look at the final TREs.
final_errors <- target_registration_errors(tx, fixed_points, moving_points)

cat(paste0('Final alignment errors in millimeters, mean(std): ',
           sprintf('%.2f',mean(final_errors)),'(',sprintf('%.2f',sd(final_errors)),') max:', sprintf('%.2f\n',max(final_errors))))

## Quantitative Evaluation II (Segmentation)

While the use of corresponding points to evaluate registration is the desired approach, it is often not applicable. In many cases there are only a few distinct points which can be localized in the two images, possibly too few to serve as a metric for evaluating the registration result across the whole region of interest. 

An alternative approach is to use segmentation. In this approach, we independently segment the structures of interest in the two images. After registration we transfer the segmentation from one image to the other and compare the original and registration induced segmentations.

In [ ]:
# Transfer the segmentation via the estimated transformation. 
# Nearest Neighbor interpolation so we don't introduce new labels.
transformed_labels <- Resample(masks[[moving_index]],
                               fixed_image,
                               tx, 
                               "sitkNearestNeighbor",
                               0.0, 
                               masks[[moving_index]]$GetPixelID())

We have now replaced the task of evaluating registration with that of evaluating segmentation.

In [ ]:
# Often referred to as ground truth, but we prefer reference as the truth is never known.
reference_segmentation <- fixed_image_mask
# Segmentations before and after registration
segmentations = c(moving_image_mask, transformed_labels == popi_lung_label)

In [ ]:
# Compute overlap related quantities.
compute_overlap_measures <- function(segmentation, reference_segmentation)
{
  # Note that for the overlap measures filter, because we are dealing with a single label we 
  # use the combined, all labels, evaluation measures without passing a specific label to the methods.
  omf <- LabelOverlapMeasuresImageFilter()
  omf$Execute(reference_segmentation, segmentation)
  result <- c(omf$GetJaccardCoefficient(), omf$GetDiceCoefficient(), 
              omf$GetVolumeSimilarity(), omf$GetFalseNegativeError(), omf$GetFalsePositiveError())
  names(result) <- c("jaccard", "dice", "volume_similarity",
                     "false_negative", "false_positive")
  return (result)
}

# Compute symmetric surface distance related quantities.
compute_surface_distance_measures <- function(segmentation, 
                                              reference_segmentation, reference_surface, num_reference_surface_pixels, reference_distance_map)
{
  segmented_label <- 1
  segmented_distance_map <- Abs(SignedMaurerDistanceMap(segmentation, FALSE, FALSE))
  segmented_surface <- LabelContour(segmentation)  
 
  # Multiply the binary surface segmentations with the distance maps. The resulting distance
  # maps contain non-zero values only on the surface (they can also contain zero on the surface)
  seg2ref_distance_map <- reference_distance_map*Cast(segmented_surface, "sitkFloat32")
  ref2seg_distance_map <- segmented_distance_map*Cast(reference_surface, "sitkFloat32")
  
  # Get the number of pixels in the segmented surface by counting all pixels that are 1.
  statistics_image_filter <- StatisticsImageFilter()    
  statistics_image_filter$Execute(segmented_surface)
  num_segmented_surface_pixels <- as.integer(statistics_image_filter$GetSum())

  # Get all non-zero distances and then add zero distances if required.
  seg2ref_distance_map_arr <- as.array(seg2ref_distance_map)
  seg2ref_distances <- seg2ref_distance_map_arr[seg2ref_distance_map_arr!=0]
  seg2ref_distances <- c(seg2ref_distances, rep(0.0, num_segmented_surface_pixels - length(seg2ref_distances)))
 
  ref2seg_distance_map_arr <- as.array(ref2seg_distance_map)
  ref2seg_distances <- ref2seg_distance_map_arr[ref2seg_distance_map_arr!=0]
  ref2seg_distances <- c(ref2seg_distances, rep(0.0, num_reference_surface_pixels - length(ref2seg_distances)))
 
  all_surface_distances = c(seg2ref_distances, ref2seg_distances)
  
  hdf = HausdorffDistanceImageFilter()  
  hdf$Execute(reference_segmentation, segmentation)
    
  result <- c(hdf$GetHausdorffDistance(),
              mean(all_surface_distances), median(all_surface_distances),
              sd(all_surface_distances), max(all_surface_distances))
  names(result) <- c("hausdorff_distance", "mean_surface_distance", "median_surface_distance", "std_surface_distance", "max_surface_distance")
  return (result)              
}

In [ ]:
overlap_df <- as.data.frame(t(sapply(segmentations, compute_overlap_measures, reference_segmentation)))
overlap_df$stage <- c("before registration", "after registration")

# Compute relevant information on reference segmentation once and then use multiple times in
# the surface based evaluation measures.
reference_distance_map <- Abs(SignedMaurerDistanceMap(reference_segmentation, FALSE, FALSE))
reference_surface <- LabelContour(reference_segmentation)
statistics_image_filter <- StatisticsImageFilter()
statistics_image_filter$Execute(reference_surface)
num_reference_surface_pixels <- as.integer(statistics_image_filter$GetSum()) 

surface_df <- as.data.frame(t(sapply(segmentations, compute_surface_distance_measures, 
                       reference_segmentation, reference_surface, num_reference_surface_pixels, reference_distance_map)))
surface_df$stage <- c("before registration", "after registration")

# Display the two data frames, to see the numbers.
overlap_df
surface_df

In [ ]:
# Use the tidyr package to organize the data frames in a column/long format for display.
overlap.gathered <- gather(overlap_df, key=overlap_measure, value=score, -stage)
surface.gathered <- gather(surface_df, key=distance_measure, value=score, -stage)
# Change the default order, alphabetical sorting, to before-after.
overlap.gathered$stage <- factor(overlap.gathered$stage, levels= c("before registration", "after registration"))
surface.gathered$stage <- factor(overlap.gathered$stage, levels= c("before registration", "after registration"))
# Display results as a bar plot.
ggplot(overlap.gathered,
       aes(x=stage, y=score, group=overlap_measure, fill=overlap_measure)) +
       geom_bar(stat="identity", position="dodge", colour='black', alpha=0.5)
ggplot(surface.gathered,
       aes(x=stage, y=score, group=distance_measure, fill=distance_measure)) +
       geom_bar(stat="identity", position="dodge", colour='black', alpha=0.5) +
       ylab("surface distance [mm]")